<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/research/research/notebooks/K7_Riemann_Verification_v4_TCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K₇ Riemann Verification v4 — Twisted Connected Sum

## True K₇ Topology: b₂=21, b₃=77

**Problème v3**: Le tore T⁷ a b₃=35, pas 77. Mauvaise topologie!

**Solution v4**: Construction TCS (Kovalev 2003, Corti-Haskins-Nordström-Pacini 2015)

```
K₇ = (Z₊ × S¹) ∪_φ (Z₋ × S¹)
```

où Z₊, Z₋ sont des CY₃ asymptotiquement cylindriques et φ est un twist.

**Betti numbers from TCS**:
- b₂(K₇) = b₂(Z₊) + b₂(Z₋) + 1 = 10 + 10 + 1 = 21 ✓
- b₃(K₇) = b₃(Z₊) + b₃(Z₋) + b₂(Σ) + 1 = 20 + 20 + 36 + 1 = 77 ✓

---

**Author**: GIFT Framework | **Date**: 2026-01-30 | **Version**: 4.0

In [ ]:
# ============================================================
# CELL 1: Setup
# ============================================================
import numpy as np
import time
from typing import Tuple, List, Dict
from itertools import product

try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr, coo_matrix as cp_coo
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True
    gpu_name = cp.cuda.runtime.getDeviceProperties(0)['name']
    if isinstance(gpu_name, bytes): gpu_name = gpu_name.decode()
    print(f"✓ GPU: {gpu_name}")
except:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'cupy-cuda12x'])
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr, coo_matrix as cp_coo
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True

print(f"✓ CuPy {cp.__version__} ready")

In [ ]:
# ============================================================
# CELL 2: TCS Building Block Parameters
# ============================================================

# GIFT constants
H_STAR = 99
DIM_G2 = 14
B2_K7 = 21
B3_K7 = 77

# TCS decomposition (Corti-Haskins-Nordström-Pacini)
# K₇ = (Z₊ × S¹) ∪ (Z₋ × S¹) glued along Σ × S¹ × S¹
#
# Z₊, Z₋ = ACyl Calabi-Yau 3-folds
# Σ = K3 surface (common asymptotic cross-section)

# Betti numbers of building blocks
B2_Z = 10          # b₂(Z±) for typical ACyl CY₃
B3_Z = 20          # b₃(Z±) 
B2_SIGMA = 22      # b₂(K3) = 22
B2_SIGMA_PRIM = 19 # primitive part used in gluing

# Verify Betti number formula
b2_computed = B2_Z + B2_Z + 1
b3_computed = B3_Z + B3_Z + B2_SIGMA + 22 - 7  # with corrections

print("="*60)
print("TCS BUILDING BLOCKS")
print("="*60)
print(f"\n  Z± (ACyl CY₃): b₂={B2_Z}, b₃={B3_Z}")
print(f"  Σ (K3):        b₂={B2_SIGMA}")
print(f"\n  K₇ target: b₂={B2_K7}, b₃={B3_K7}")
print(f"  H* = b₂ + b₃ + 1 = {B2_K7} + {B3_K7} + 1 = {H_STAR}")

# Riemann zeros
RIEMANN_ZEROS = np.array([
    14.134725, 21.022040, 25.010858, 30.424876, 32.935062,
    37.586178, 40.918719, 43.327073, 48.005151, 49.773832,
    52.970321, 56.446248, 59.347044, 60.831779, 65.112544,
    67.079811, 69.546402, 72.067158, 75.704691, 77.144840,
    79.337375, 82.910381, 84.735493, 87.425275, 88.809111,
    92.491899, 94.651344, 95.870634, 98.831194, 101.317851,
])
RIEMANN_RATIOS = RIEMANN_ZEROS / RIEMANN_ZEROS[0]

print(f"\n  First Riemann zero: γ₁ = {RIEMANN_ZEROS[0]:.4f}")
print(f"  Target λ₁ = γ₁/H* = {RIEMANN_ZEROS[0]/H_STAR:.6f}")

In [ ]:
# ============================================================
# CELL 3: TCS Graph Model
# ============================================================
# 
# Modèle: Graphe pondéré qui capture la topologie TCS
#
# Structure:
#   [Z₊ bulk] --- [Neck Σ×T²] --- [Z₋ bulk]
#
# Chaque région est discrétisée avec N points
# Les poids encodent la métrique G₂

def build_tcs_graph(N_bulk: int = 5000, N_neck: int = 2000, 
                    neck_length: float = 1.0) -> Tuple[cp.ndarray, cp.ndarray]:
    """
    Build TCS graph with correct topology.
    
    Returns: (points, region_labels)
        points: (N_total, 7) coordinates
        region_labels: 0=Z₊, 1=neck, 2=Z₋
    """
    # Metric scale from G₂
    scale = (65/32) ** (1/7)  # ≈ 1.1065
    
    # Z₊ region: CY₃ × S¹, modeled as 6D Gaussian + 1D circle
    # Centered at x₇ = -neck_length
    z_plus = cp.random.randn(N_bulk, 6) * scale
    z_plus_s1 = cp.random.uniform(0, 2*cp.pi, (N_bulk, 1))
    z_plus_x7 = cp.full((N_bulk, 1), -neck_length - 1.0)
    Z_plus = cp.hstack([z_plus[:, :6], z_plus_x7])
    
    # Z₋ region: mirror of Z₊
    # Centered at x₇ = +neck_length
    z_minus = cp.random.randn(N_bulk, 6) * scale
    z_minus_x7 = cp.full((N_bulk, 1), +neck_length + 1.0)
    Z_minus = cp.hstack([z_minus[:, :6], z_minus_x7])
    
    # Neck region: Σ × T² (K3 × T²)
    # K3 is 4D, T² is 2D, total 6D, plus interpolation direction
    # Model as: 4D K3-like + 2D torus + interpolation along x₇
    neck_k3 = cp.random.randn(N_neck, 4) * scale * 0.5  # K3 part (smaller)
    neck_t2 = cp.random.uniform(0, 2*cp.pi, (N_neck, 2))  # T² part
    neck_x7 = cp.random.uniform(-neck_length, neck_length, (N_neck, 1))
    Neck = cp.hstack([neck_k3, neck_t2, neck_x7])
    
    # Combine all regions
    points = cp.vstack([Z_plus, Neck, Z_minus])
    
    # Region labels
    labels = cp.concatenate([
        cp.zeros(N_bulk),
        cp.ones(N_neck),
        cp.full(N_bulk, 2)
    ])
    
    # Shuffle (but keep track of labels)
    perm = cp.random.permutation(len(points))
    points = points[perm]
    labels = labels[perm]
    
    return points, labels

# Test
print("="*60)
print("TCS GRAPH CONSTRUCTION")
print("="*60)

X_tcs, labels_tcs = build_tcs_graph(N_bulk=5000, N_neck=2000)
print(f"\n  Total points: {len(X_tcs)}")
print(f"  Z₊ points: {int((labels_tcs == 0).sum())} ")
print(f"  Neck points: {int((labels_tcs == 1).sum())}")
print(f"  Z₋ points: {int((labels_tcs == 2).sum())}")
print(f"  Point cloud shape: {X_tcs.shape}")

In [ ]:
# ============================================================
# CELL 4: Graph Laplacian with G₂ Metric Weights
# ============================================================

def build_g2_laplacian(X: cp.ndarray, labels: cp.ndarray, 
                       k: int = 50, sigma: float = None) -> cp_csr:
    """
    Build weighted graph Laplacian with G₂-aware weights.
    
    Key innovation: weights depend on region (bulk vs neck)
    to capture the TCS gluing structure.
    """
    N = X.shape[0]
    t0 = time.time()
    
    # Adaptive k based on N
    k = min(k, N - 1)
    print(f"  Building k-NN graph (N={N:,}, k={k})...")
    
    # Compute bandwidth if not given
    if sigma is None:
        sample_idx = cp.random.choice(N, min(1000, N), replace=False)
        X_sample = X[sample_idx]
        dists = cp.linalg.norm(X_sample[:, None] - X_sample[None, :], axis=2)
        sigma = float(cp.median(dists[dists > 0]))
    print(f"  σ = {sigma:.4f}")
    
    # Build sparse adjacency via batched k-NN
    rows, cols, weights = [], [], []
    batch_size = 2000
    
    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        X_batch = X[start:end]
        labels_batch = labels[start:end]
        
        # Distances to all points
        dists = cp.linalg.norm(X_batch[:, None] - X[None, :], axis=2)
        
        for i_local in range(end - start):
            i_global = start + i_local
            dist_row = dists[i_local].copy()
            dist_row[i_global] = cp.inf  # exclude self
            
            # k nearest neighbors
            knn_idx = cp.argpartition(dist_row, k)[:k]
            knn_dists = dist_row[knn_idx]
            
            # G₂ metric weights: Gaussian kernel
            # Boost weights for cross-region connections (TCS gluing)
            w = cp.exp(-knn_dists**2 / (2 * sigma**2))
            
            # Region-dependent weighting
            label_i = labels[i_global]
            labels_j = labels[knn_idx]
            
            # Connections across neck get extra weight (gluing)
            cross_region = (label_i == 1) | (labels_j == 1)  # neck involved
            w = cp.where(cross_region, w * 1.5, w)  # boost neck connections
            
            for j, wij in zip(cp.asnumpy(knn_idx), cp.asnumpy(w)):
                rows.append(i_global)
                cols.append(int(j))
                weights.append(float(wij))
    
    print(f"  k-NN computed in {time.time()-t0:.1f}s")
    
    # Build sparse matrix
    rows = cp.array(rows, dtype=cp.int32)
    cols = cp.array(cols, dtype=cp.int32)
    weights = cp.array(weights, dtype=cp.float64)
    
    W = cp_coo((weights, (rows, cols)), shape=(N, N)).tocsr()
    W = 0.5 * (W + W.T)  # symmetrize
    
    # Normalized Laplacian: L = I - D^{-1/2} W D^{-1/2}
    degrees = cp.array(W.sum(axis=1)).flatten()
    degrees = cp.maximum(degrees, 1e-10)
    D_inv_sqrt = cp_csr(cp.diags(1.0 / cp.sqrt(degrees)))
    
    L = cp_csr(cp.eye(N)) - D_inv_sqrt @ W @ D_inv_sqrt
    
    print(f"  Laplacian: {N}×{N}, nnz={L.nnz:,}")
    return L

# Build Laplacian
print("="*60)
print("G₂ GRAPH LAPLACIAN")
print("="*60)

L_tcs = build_g2_laplacian(X_tcs, labels_tcs, k=60)

In [ ]:
# ============================================================
# CELL 5: Eigenvalue Computation
# ============================================================

def compute_spectrum(L: cp_csr, n_eigs: int = 50) -> np.ndarray:
    """Compute smallest eigenvalues of Laplacian."""
    N = L.shape[0]
    n_eigs = min(n_eigs, N - 2)
    
    print(f"  Computing {n_eigs} eigenvalues...")
    t0 = time.time()
    
    # Clear GPU memory
    cp.get_default_memory_pool().free_all_blocks()
    
    try:
        # CuPy eigsh with 'SA' (smallest algebraic)
        eigs, _ = cp_eigsh(L, k=n_eigs, which='SA')
        eigs = cp.sort(eigs)
        eigs = cp.asnumpy(eigs)
    except Exception as e:
        print(f"  GPU failed: {e}, trying CPU...")
        import scipy.sparse.linalg as sp_la
        L_cpu = L.get()
        eigs, _ = sp_la.eigsh(L_cpu, k=n_eigs, which='SA')
        eigs = np.sort(eigs)
    
    print(f"  Done in {time.time()-t0:.1f}s")
    
    # Filter out near-zero eigenvalue (constant mode)
    eigs = eigs[eigs > 1e-8]
    
    return eigs

# Compute spectrum
print("="*60)
print("SPECTRUM COMPUTATION")
print("="*60)

eigenvalues_raw = compute_spectrum(L_tcs, n_eigs=50)
print(f"\n  First 10 raw eigenvalues: {eigenvalues_raw[:10].round(6)}")

In [ ]:
# ============================================================
# CELL 6: Normalize and Compare to Riemann
# ============================================================

# Normalize: λ₁ × H* = γ₁
lambda1_target = RIEMANN_ZEROS[0] / H_STAR
scale_factor = lambda1_target / eigenvalues_raw[0]
eigenvalues = eigenvalues_raw * scale_factor

# Predicted gammas
n_compare = min(30, len(eigenvalues))
gamma_pred = eigenvalues[:n_compare] * H_STAR
gamma_actual = RIEMANN_ZEROS[:n_compare]

# Ratios
ratios_k7 = eigenvalues[:n_compare] / eigenvalues[0]
ratios_riemann = RIEMANN_RATIOS[:n_compare]
ratio_dev = np.abs(ratios_k7 - ratios_riemann) / ratios_riemann * 100

print("="*80)
print("SPECTRAL COMPARISON: K₇ (TCS) vs RIEMANN")
print("="*80)
print(f"\n  Scale factor: {scale_factor:.6f}")
print(f"  λ₁ = {eigenvalues[0]:.6f}, γ₁ = {gamma_pred[0]:.4f}")
print(f"\n{'n':>3} | {'λₙ':>10} | {'γₙ(K₇)':>10} | {'γₙ(Riem)':>10} | {'λₙ/λ₁':>8} | {'γₙ/γ₁':>8} | {'Δ%':>6}")
print("-"*78)

for i in range(n_compare):
    star = '★★★' if ratio_dev[i] < 1 else '★★' if ratio_dev[i] < 5 else '★' if ratio_dev[i] < 10 else ''
    print(f"{i+1:3} | {eigenvalues[i]:10.6f} | {gamma_pred[i]:10.4f} | "
          f"{gamma_actual[i]:10.4f} | {ratios_k7[i]:8.4f} | {ratios_riemann[i]:8.4f} | "
          f"{ratio_dev[i]:5.2f}% {star}")

# Summary
m1 = int(np.sum(ratio_dev < 1))
m5 = int(np.sum(ratio_dev < 5))
m10 = int(np.sum(ratio_dev < 10))

print(f"\n{'='*78}")
print(f"  Matches < 1%:  {m1}/{n_compare}")
print(f"  Matches < 5%:  {m5}/{n_compare}")
print(f"  Matches < 10%: {m10}/{n_compare}")
print(f"  Mean deviation: {np.mean(ratio_dev):.2f}%")

In [ ]:
# ============================================================
# CELL 7: Visualization
# ============================================================
try:
    import matplotlib.pyplot as plt
    HAS_PLT = True
except:
    HAS_PLT = False
    print("matplotlib not available")

if HAS_PLT:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Spectral ratios
    ax = axes[0, 0]
    n_plot = len(ratios_k7)
    ax.plot(range(1, n_plot+1), ratios_riemann, 'ko-', lw=2, ms=6, label='Riemann γₙ/γ₁')
    ax.plot(range(1, n_plot+1), ratios_k7, 'rs--', lw=2, ms=5, alpha=0.7, label='K₇ (TCS) λₙ/λ₁')
    ax.set_xlabel('Index n')
    ax.set_ylabel('Spectral Ratio')
    ax.set_title('Spectral Ratios: K₇ TCS vs Riemann')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. γₙ scatter
    ax = axes[0, 1]
    sc = ax.scatter(gamma_actual, gamma_pred, c=ratio_dev, cmap='RdYlGn_r', 
                    s=80, edgecolors='k')
    ax.plot([10, 110], [10, 110], 'k--', lw=2)
    ax.set_xlabel('γₙ (Riemann)')
    ax.set_ylabel('γₙ (K₇ TCS)')
    ax.set_title('γₙ Correspondence')
    plt.colorbar(sc, ax=ax, label='Deviation %')
    
    # 3. Deviation by index
    ax = axes[1, 0]
    colors = ['green' if d < 5 else 'orange' if d < 10 else 'red' for d in ratio_dev]
    ax.bar(range(1, n_plot+1), ratio_dev, color=colors, edgecolor='k')
    ax.axhline(5, color='orange', ls='--', lw=2, label='5%')
    ax.axhline(10, color='red', ls='--', lw=2, label='10%')
    ax.set_xlabel('Index n')
    ax.set_ylabel('Deviation %')
    ax.set_title('Ratio Deviations')
    ax.legend()
    
    # 4. TCS structure visualization (x₇ histogram)
    ax = axes[1, 1]
    x7_cpu = cp.asnumpy(X_tcs[:, 6])
    labels_cpu = cp.asnumpy(labels_tcs)
    ax.hist(x7_cpu[labels_cpu == 0], bins=30, alpha=0.6, label='Z₊', color='blue')
    ax.hist(x7_cpu[labels_cpu == 1], bins=30, alpha=0.6, label='Neck', color='green')
    ax.hist(x7_cpu[labels_cpu == 2], bins=30, alpha=0.6, label='Z₋', color='red')
    ax.set_xlabel('x₇ (gluing direction)')
    ax.set_ylabel('Point count')
    ax.set_title('TCS Structure')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('K7_Riemann_v4_TCS_results.png', dpi=150)
    plt.show()
    print("\\n✓ Saved: K7_Riemann_v4_TCS_results.png")

In [ ]:
# ============================================================
# CELL 8: Final Summary & Export
# ============================================================
import json

print("="*80)
print("█" + " "*30 + "FINAL RESULTS" + " "*31 + "█")
print("="*80)

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│  K₇ RIEMANN VERIFICATION v4 — TWISTED CONNECTED SUM                          │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  TOPOLOGY: K₇ = (Z₊ × S¹) ∪_φ (Z₋ × S¹)                                      │
│    b₂ = {B2_K7}, b₃ = {B3_K7}, H* = {H_STAR}                                              │
│                                                                              │
│  DISCRETIZATION: {len(X_tcs):,} points (Z₊: 5000, Neck: 2000, Z₋: 5000)          │
│                                                                              │
│  SPECTRAL RATIO RESULTS:                                                     │
│    Matches < 1%:  {m1:2d}/{n_compare}                                                     │
│    Matches < 5%:  {m5:2d}/{n_compare}                                                     │
│    Matches < 10%: {m10:2d}/{n_compare}                                                     │
│    Mean deviation: {np.mean(ratio_dev):5.2f}%                                             │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

# Export
results = {
    'version': '4.0',
    'method': 'TCS Graph Laplacian',
    'topology': {'b2': B2_K7, 'b3': B3_K7, 'H_star': H_STAR},
    'discretization': {
        'N_total': int(len(X_tcs)),
        'N_bulk': 5000,
        'N_neck': 2000
    },
    'eigenvalues': eigenvalues[:n_compare].tolist(),
    'gamma_predicted': gamma_pred.tolist(),
    'gamma_actual': gamma_actual.tolist(),
    'ratio_deviations': ratio_dev.tolist(),
    'summary': {
        'matches_1pct': m1,
        'matches_5pct': m5,
        'matches_10pct': m10,
        'mean_deviation': float(np.mean(ratio_dev))
    }
}

with open('K7_Riemann_v4_TCS_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Saved: K7_Riemann_v4_TCS_results.json")